In [232]:
import connections

In [233]:
path_to_candidate_dataset = r"test_data/dataset1/candidate_dataset.csv"
path_to_target_dataset = r"test_data/dataset1/target_dataset.csv"

In [234]:
# Open datasets, concat everything into a single field and add to db
import pandas as pd

In [235]:
df_candidate = pd.read_csv(path_to_candidate_dataset, encoding="utf-8",  dtype=object,)
df_target = pd.read_csv(path_to_target_dataset, encoding="utf-8",  dtype=object,)

In [236]:
from data_preprocessing import add_dmetaphone_cols
df_candidate = add_dmetaphone_cols(df_candidate, ["first_name", "surname", "city"])
df_target = add_dmetaphone_cols(df_target, ["first_name", "surname", "city"])

In [237]:
# Add unique identifiers to these datasets
def add_unique_id(df,prefix=""):
    df["auto_generated_row_id"] = 1
    df["auto_generated_row_id"] = df["auto_generated_row_id"].cumsum()
    df["auto_generated_row_id"] = prefix + df["auto_generated_row_id"].astype(str)
    return df

In [238]:
df_candidate = add_unique_id(df_candidate, "candidate_")
df_target = add_unique_id(df_target, "target_")

In [239]:
def concat_fields(df, drop_list = []):
    
    for r in df.iterrows():
        index = r[0]
        row = r[1]
        
        concats = []
        
        for k in row.keys():
            if k not in drop_list:
                if row[k] and not pd.isnull(row[k]):
                    concats.append(str(row[k]))
        
        df.loc[index, "concat_all"] = " ".join(concats)
        
    return df

In [240]:
df_target = concat_fields(df_target, drop_list=["code","auto_generated_row_id"])
df_candidate = concat_fields(df_candidate, drop_list=["code","auto_generated_row_id"])

In [241]:
df_target.head()

,gender,first_name,surname,city,code,year,month,day,first_name_dmetaphone,surname_dmetaphone,city_dmetaphone,auto_generated_row_id,concat_all
0,Male,Matt,Hughes,Kirklees,a0f7d1e3def8a22d2708479d3e14a1e8,2003,Jun,11,MT,HS,KRKL,target_1,Male Matt Hughes Kirklees 2003 Jun 11 MT HS KRKL
1,Male,Tom,Brown,Birmingham,0bd7766ac3568d300d6c11e1369021e4,2006,Jan,19,TM,PRN,PRMN,target_2,Male Tom Brown Birmingham 2006 Jan 19 TM PRN PRMN
2,Male,Tom,Hughes,Bristol,12693dc197b8e29185c8c8fb847599fa,1997,Jun,21,TM,HS,PRST,target_3,Male Tom Hughes Bristol 1997 Jun 21 TM HS PRST
3,Male,Alex,Wilson,Birmingham,2ec6b298ee37161204f187e32a1981b0,2012,Jan,20,ALKS,ALSN,PRMN,target_4,Male Alex Wilson Birmingham 2012 Jan 20 ALKS A...
4,Male,John,Walker,Bradford,8be9a41303d2f52784d4c55a1561064a,1993,Sep,29,JN,ALKR,PRTF,target_5,Male John Walker Bradford 1993 Sep 29 JN ALKR ...


In [242]:
#Data for postgres
df_target_for_db = df_target[["auto_generated_row_id", "concat_all"]]
df_candidate_for_db = df_candidate[["auto_generated_row_id", "concat_all"]]

In [243]:
#Put in sqlite (do we actually need postgres for small tables?)
import sqlite3

In [244]:
con = sqlite3.connect(":memory:") 

In [245]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://') 

In [246]:
con = engine.connect()

In [247]:
df_target_for_db.to_sql("target", engine, index=False)

In [249]:
result = con.execute("CREATE VIRTUAL TABLE fts_target USING fts4(auto_generated_row_id TEXT, concat_all TEXT);")

In [250]:
result = con.execute("INSERT INTO fts_target SELECT * FROM target")

In [251]:
pd.read_sql("select * from fts_target where concat_all MATCH 'Matt'", con)

,auto_generated_row_id,concat_all
0,target_1,Male Matt Hughes Kirklees 2003 Jun 11 MT HS KRKL
1,target_9,Male Matt Wright Leeds 2011 Sep 05 MT RT LTS
2,target_12,Male Matt Hughes Leeds 2000 Apr 22 MT HS LTS
3,target_14,Male Matt Robinson Fife 1995 Jun 08 MT RPNS FF
4,target_20,Male Matt Walker Bradford 1989 Feb 23 MT ALKR ...
5,target_23,Male Matt Evans Leeds 1995 Aug 24 MT AFNS LTS
6,target_25,Male Matt Hughes Bradford 2011 Feb 07 MT HS PRTF
7,target_33,Male Matt Wood Glasgow 2014 Aug 17 MT AT KLSK
8,target_47,Male Matt Walker Birmingham 1998 Mar 14 MT ALK...
9,target_60,Male Matt Martin Kirklees 2009 Apr 18 MT MRTN ...


In [252]:
sql = """
create virtual table terms using fts4aux(fts_target);

"""
con.execute(sql)

In [271]:
from future_builtins import map  # Only on Python 2

from collections import Counter
from itertools import chain

results = con.execute("select concat_all from target")
def row_to_str_split(r):
    return r[0].split()

freq_counts = Counter(chain.from_iterable(map(row_to_str_split, results)))
total_token_count = sum(freq_counts.values())

In [273]:
{k: freq_counts[k]*1.0/total_token_count for k in freq_counts}

{u'01': 0.002,
 u'02': 0.004,
 u'04': 0.002,
 u'05': 0.007,
 u'06': 0.004,
 u'07': 0.003,
 u'08': 0.007,
 u'09': 0.003,
 u'10': 0.002,
 u'11': 0.002,
 u'12': 0.003,
 u'13': 0.003,
 u'14': 0.005,
 u'15': 0.001,
 u'16': 0.003,
 u'17': 0.007,
 u'18': 0.004,
 u'19': 0.003,
 u'1988': 0.006,
 u'1989': 0.003,
 u'1990': 0.002,
 u'1991': 0.001,
 u'1992': 0.003,
 u'1993': 0.003,
 u'1994': 0.004,
 u'1995': 0.006,
 u'1996': 0.001,
 u'1997': 0.004,
 u'1998': 0.003,
 u'1999': 0.001,
 u'20': 0.004,
 u'2000': 0.002,
 u'2001': 0.003,
 u'2002': 0.004,
 u'2003': 0.004,
 u'2004': 0.003,
 u'2005': 0.002,
 u'2006': 0.005,
 u'2007': 0.004,
 u'2008': 0.01,
 u'2009': 0.004,
 u'2010': 0.004,
 u'2011': 0.004,
 u'2012': 0.004,
 u'2013': 0.003,
 u'2014': 0.003,
 u'2015': 0.001,
 u'2016': 0.003,
 u'21': 0.005,
 u'22': 0.004,
 u'23': 0.002,
 u'24': 0.002,
 u'25': 0.001,
 u'26': 0.003,
 u'27': 0.004,
 u'28': 0.002,
 u'29': 0.003,
 u'30': 0.001,
 u'31': 0.004,
 u'AFNS': 0.007,
 u'ALFR': 0.012,
 u'ALKR': 0.009,
 u'ALKS

In [263]:
sum(freq_counts.values())

1000

In [264]:
freq_counts.values()

[11,
 9,
 7,
 12,
 11,
 5,
 1,
 10,
 5,
 6,
 3,
 4,
 4,
 5,
 4,
 2,
 7,
 6,
 2,
 3,
 12,
 1,
 3,
 9,
 3,
 4,
 4,
 7,
 4,
 1,
 2,
 10,
 3,
 6,
 3,
 1,
 4,
 4,
 3,
 6,
 9,
 7,
 3,
 1,
 8,
 20,
 6,
 3,
 5,
 5,
 5,
 2,
 11,
 4,
 2,
 11,
 2,
 4,
 3,
 2,
 7,
 11,
 7,
 3,
 6,
 13,
 10,
 2,
 2,
 4,
 1,
 6,
 6,
 7,
 80,
 3,
 7,
 12,
 4,
 1,
 11,
 2,
 8,
 3,
 7,
 12,
 7,
 6,
 9,
 16,
 1,
 5,
 10,
 3,
 4,
 13,
 3,
 10,
 6,
 12,
 9,
 3,
 3,
 2,
 2,
 3,
 3,
 6,
 5,
 4,
 4,
 3,
 4,
 5,
 9,
 7,
 3,
 3,
 1,
 10,
 12,
 5,
 2,
 6,
 5,
 2,
 12,
 6,
 7,
 9,
 10,
 10,
 7,
 16,
 10,
 2,
 4,
 2,
 4,
 5,
 10,
 2,
 2,
 4,
 3,
 5,
 3,
 1,
 9,
 10,
 3,
 4,
 4,
 2,
 3,
 5,
 4,
 3,
 10,
 4,
 10,
 4,
 6,
 4]